In [1]:
import pandas as pd
import traintools
from trainconstants import *
from collections import defaultdict
import networkx as nx
from typing import List, Tuple

TrainStop = Tuple[int, int]

In [2]:
def attempt_solution_1(G: nx.DiGraph) -> Tuple[int, defaultdict, defaultdict]:
    """Here is a first attempt at a solution for trains of only one type.

    The solution works a follows:
    * start by finding the longest path in the graph
    * next go through that longest path and subtract the 
    maximum number of passengers that can fit in the train on each
    part of the trip
    * recalculate a new longest path in the altered graph and repeat until the 
    longest path consists of only one node.
    * meanwhile keep track of how many trains were needed and at which stations the trains start and end.

    Known problems:
    * The algorithm will think that it is "better" to choose a trip where there are 1000
    passengers to be transported than to choose a trip where there are 500 passengers to be transported
    eventhough both exeed the capacity of the train so they would both be equilly valid options.
    * the number of trains starting and ending at each station is not the same 
    * the solution is incorrect because of the point mentioned above.
    """

    def do_bookkeeping(path: List[TrainStop]) -> None:
        """Update the variables that keep track of the total number of trains 
        and where they start and end.
        """
        nonlocal starting_trains, ending_trains, number_of_trains

        starting_trains[path[0][0]].append(path)
        ending_trains[path[-1][0]].append(path)
        number_of_trains += 1

    # set up variables to keep track of the number of trains and at which stations the trains start and end.
    starting_trains, ending_trains, number_of_trains = defaultdict(list), defaultdict(list), 0

    # find the longest path in the graph
    path = nx.dag_longest_path(G, weight='second_class')

    # repeat until the longest path consists of only one node
    while len(path) != 1:
        do_bookkeeping(path)
        # go through the longest path and pick up the maximal number of passengers on the way
        for index, current_stop in enumerate(path[:-1]):
            next_stop = path[index + 1]
            G[current_stop][next_stop]['first_class'] -= min(TYPE_3_TRAIN[0], G[current_stop][next_stop]['first_class'])
            G[current_stop][next_stop]['second_class'] -= min(TYPE_3_TRAIN[1], G[current_stop][next_stop]['second_class'])
        
        # recompute the longest path 
        path = nx.dag_longest_path(G, weight='second_class')

    return (
        number_of_trains, 
        {key: len(value) for key, value in starting_trains.items()},
        {key: len(value) for key, value in ending_trains.items()}
        )

In [3]:
df = traintools.read_schedule("datasets/nsdata1.txt")
G = traintools.graph_from_schedule(df)


print(attempt_solution_1(G.copy()))

(24, {4: 7, 2: 13, 3: 4}, {3: 5, 2: 4, 1: 5, 4: 10})


In [4]:
def attempt_solution_2(G: nx.DiGraph) -> Tuple[int, defaultdict, defaultdict]:
    """Here is a second attempt, it is very similar but tries to fix the endpoints.

    This is done through the use of the fact that nx.dag_longest_path returns "the shortest" 
    longest path. Meaning that when the weight of the edges is 0, it does not add any of those edges 
    to the end of the longest path 
    (investigate: does it add 'unnecessary edges to the front?')
    This means that we can try to
    add (all the trains that do not pick up passengers everywhere) empty trips to the front and the back of 
    these trains and see if we can connect them in a way such that the number of trains at each station 
    at the beginning of the day is equal to the number of trains at the end of the day. 

    Note:
    * Did not actually implement this yet, as of now this is pretty much the same as solution above.
    """

    def do_bookkeeping(path: List[TrainStop]) -> None:
        """Update the variables that keep track of the total number of trains 
        and where they start and end.
        """
        nonlocal starting_trains, ending_trains, number_of_trains

        starting_trains[path[0][0]].append(path)
        ending_trains[path[-1][0]].append(path)
        number_of_trains += 1

    # set up variables to keep track of the number of trains and at which stations the trains start and end.
    starting_trains, ending_trains, number_of_trains = defaultdict(list), defaultdict(list), 0

    # find the longest path in the graph
    path = nx.dag_longest_path(G, weight='second_class')

    # repeat until the longest path consists of only one node
    while len(path) != 1:
        do_bookkeeping(path)
        # go through the longest path and pick up the maximal number of passengers on the way
        for index, current_stop in enumerate(path[:-1]):
            next_stop = path[index + 1]
            G[current_stop][next_stop]['first_class'] -= min(TYPE_3_TRAIN[0], G[current_stop][next_stop]['first_class'])
            G[current_stop][next_stop]['second_class'] -= min(TYPE_3_TRAIN[1], G[current_stop][next_stop]['second_class'])
        
        # recompute the longest path 
        path = nx.dag_longest_path(G, weight='second_class')

    return (
        number_of_trains, 
        starting_trains, ending_trains
        )

In [5]:
number_of_trains, starting_trains, ending_trains = attempt_solution_2(G.copy())

In [6]:
for paths in starting_trains.values():
    for path in paths:
        print(path)

[(4, 330), (4, 414), (3, 468), (3, 472), (2, 512), (2, 515), (2, 538), (2, 542), (3, 581), (3, 583), (3, 590), (3, 593), (2, 632), (2, 634), (1, 698), (1, 716), (2, 778), (2, 782), (3, 821), (3, 823), (3, 830), (3, 833), (2, 872), (2, 875), (1, 938), (1, 956), (2, 1018), (2, 1021), (3, 1063), (3, 1065), (3, 1070), (3, 1073), (2, 1112), (2, 1114), (2, 1138), (2, 1142), (3, 1181), (3, 1183), (3, 1190), (3, 1193), (2, 1232), (2, 1235), (1, 1298), (1, 1316), (2, 1378), (2, 1382), (3, 1434)]
[(4, 330), (3, 395), (3, 403), (2, 446), (2, 452), (1, 518), (1, 536), (2, 598), (2, 603), (3, 643), (3, 645), (3, 650), (3, 653), (2, 692), (2, 694), (1, 758), (1, 776), (2, 838), (2, 842), (3, 881), (3, 883), (3, 890), (3, 893), (2, 932), (2, 934), (2, 958), (2, 960), (3, 1003), (3, 1005), (3, 1010), (3, 1013), (2, 1053), (2, 1055), (1, 1118), (1, 1136), (2, 1198), (2, 1202), (3, 1241), (3, 1243), (3, 1249), (3, 1252), (2, 1290), (2, 1292), (1, 1358), (1, 1376), (2, 1438)]
[(4, 330), (4, 414), (3, 468

In [7]:
starting_stops = tuple(traintools.find_starting_trainstops(G))
ending_stops = tuple(traintools.find_ending_trainstops(G))

print(starting_stops)

for paths in starting_trains.values():
    for path in paths:
        print(sum(1 for trainstop in starting_stops if nx.has_path(G, trainstop, path[0])))

((2, 331), (3, 329), (4, 330), (1, 399))
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1


In [8]:
for paths in ending_trains.values():
    for path in paths:
        print(sum(1 for trainstop in ending_stops if nx.has_path(G, path[-1], trainstop)))

1
1
4
4
4
1
2
4
4
1
4
3
4
4
1
1
1
4
1
1
4
4
4
4


In [9]:
# 0.
# Create a list of lists (or tuple of tuples, or some combination) that can be searched by using backtracking 
# to find a solution in which the endpoints are fixed.

In [ ]:
# 1.
# Create a function that determines wheter it is sufficient
# to weight by only second class passengers. 
# I.e. a function that determines the bottle neck when calculation the 
# minimum number of trains on each trip

In [ ]:
# 2.
# Switch the visualization to plot from nx.DiGraph instead
# of plotting from pd.DataFrame, this will enable fast checking of 
# intuitive statements about what the train trajectories look like.